In [1]:
from ansatzes.sim1 import Sim1
from ansatzes.sim14 import Sim14
from generators import get_ansatz_generators

print("=" * 60)
print("Testing Sim1 (single qubit gates only)")
print("=" * 60)
ansatz1 = Sim1(num_qubits=4, depth=2)
generators1 = get_ansatz_generators(ansatz1)
print(f"Total generators: {len(generators1)}\n")
for gen, wires, name in generators1[:5]:  # Show first 5
    print(f"{name} on wires {wires.tolist()}: {gen}")

print("\n" + "=" * 60)
print("Testing Sim14 (single + two qubit gates)")
print("=" * 60)
ansatz14 = Sim14(num_qubits=4, depth=1)
generators14 = get_ansatz_generators(ansatz14)
print(f"Total generators: {len(generators14)}\n")
for gen, wires, name in generators14:
    print(f"{name} on wires {wires.tolist()}: {gen.matrix()}")

Testing Sim1 (single qubit gates only)
Total generators: 16

RX on wires [0]: -0.5 * X(0)
RZ on wires [0]: -0.5 * Z(0)
RX on wires [1]: -0.5 * X(1)
RZ on wires [1]: -0.5 * Z(1)
RX on wires [2]: -0.5 * X(2)

Testing Sim14 (single + two qubit gates)
Total generators: 16

RY on wires [0]: [[0.+0.j  0.+0.5j]
 [0.-0.5j 0.+0.j ]]
RY on wires [1]: [[0.+0.j  0.+0.5j]
 [0.-0.5j 0.+0.j ]]
RY on wires [2]: [[0.+0.j  0.+0.5j]
 [0.-0.5j 0.+0.j ]]
RY on wires [3]: [[0.+0.j  0.+0.5j]
 [0.-0.5j 0.+0.j ]]
CRX on wires [3, 0]: [[ 0. +0.j -0. +0.j  0. +0.j -0. +0.j]
 [-0. +0.j  0. +0.j -0. +0.j  0. +0.j]
 [ 0. +0.j -0. +0.j  0. +0.j -0.5+0.j]
 [-0. +0.j  0. +0.j -0.5+0.j  0. +0.j]]
CRX on wires [2, 3]: [[ 0. +0.j -0. +0.j  0. +0.j -0. +0.j]
 [-0. +0.j  0. +0.j -0. +0.j  0. +0.j]
 [ 0. +0.j -0. +0.j  0. +0.j -0.5+0.j]
 [-0. +0.j  0. +0.j -0.5+0.j  0. +0.j]]
CRX on wires [1, 2]: [[ 0. +0.j -0. +0.j  0. +0.j -0. +0.j]
 [-0. +0.j  0. +0.j -0. +0.j  0. +0.j]
 [ 0. +0.j -0. +0.j  0. +0.j -0.5+0.j]
 [-0. +0.j  

In [2]:
# Create symmetric group S_4
from symmetry_groups import create_induced_subgroup, create_symmetric_group, verify_subgroup_closure


S4 = create_symmetric_group(4)

# Choose some generators (e.g., transposition (0,1) and 4-cycle (0,1,2,3))
# Find their indices in S4
generators = []
for idx, elem in enumerate(S4.elements):
    if elem == (1, 0, 3, 2):  # transposition
        generators.append(idx)
    if elem == (2, 1, 0, 3):  # 4-cycle
        generators.append(idx)
    if elem == (0, 1, 2, 3):  # another transposition
        generators.append(idx)

# Create induced subgroup
H = create_induced_subgroup(S4, generators)

# Verify it's a proper subgroup
print(f"S4 has {len(S4)} elements")
print(f"Induced subgroup H has {len(H)} elements")
print(f"H satisfies closure: {verify_subgroup_closure(H)}")

Induced subgroup has 8 elements (generated from 3 generators in 3 iterations)
S4 has 24 elements
Induced subgroup H has 8 elements
H satisfies closure: True


In [3]:
# Find Klein-4 elements in S4
klein4_generators = []
for idx, elem in enumerate(S4.elements):
    if elem == (0, 1, 2, 3):  # identity
        klein4_generators.append(idx)
    if elem == (1, 0, 3, 2):  # (1,2)(3,4)
        klein4_generators.append(idx)
    if elem == (2, 3, 0, 1):  # (1,3)(2,4)
        klein4_generators.append(idx)
    if elem == (3, 2, 1, 0):  # (1,4)(2,3)
        klein4_generators.append(idx)

K4 = create_induced_subgroup(S4, klein4_generators)

Induced subgroup has 4 elements (generated from 4 generators in 1 iterations)


In [4]:
for idx, elem in enumerate(K4.elements):
    print(f"Element {idx}: {elem}")
    print(H.get_matrix(idx))

Element 0: (0, 1, 2, 3)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Element 1: (1, 0, 3, 2)
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
Element 2: (2, 3, 0, 1)
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
Element 3: (3, 2, 1, 0)
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [5]:
import numpy as np


def get_angle_encoding_unitary(data):
    """
    Get the unitary matrix U(x) for AngleEmbedding.
    
    For AngleEmbedding with rotation='Z':
    U(x) = ⨂_{i=0}^{n-1} RZ(x[i])
    
    where RZ(θ) = [[e^(-iθ/2), 0], [0, e^(iθ/2)]]
    """
    # Start with 1x1 identity
    U = np.array([[1.0]], dtype=complex)
    
    for i in range(len(data)):
        # RZ rotation for qubit i
        RZ = np.array([
            [np.exp(-1j * data[i] / 2), 0],
            [0, np.exp(1j * data[i] / 2)]
        ], dtype=complex)
        
        # Tensor product
        U = np.kron(U, RZ)
    
    return U

def angle_encoding_unitary_mixed(x):
    # Example: per qubit RZ(x_i) RX(x_i) RY(x_i)
    U = np.array([[1.+0j]])
    for theta in x:
        RZ = np.array([[np.exp(-1j*theta/2), 0],
                       [0, np.exp(1j*theta/2)]], dtype=complex)
        RX = np.array([[np.cos(theta/2), -1j*np.sin(theta/2)],
                       [-1j*np.sin(theta/2), np.cos(theta/2)]], dtype=complex)
        RY = np.array([[np.cos(theta/2), -np.sin(theta/2)],
                       [np.sin(theta/2),  np.cos(theta/2)]], dtype=complex)
        # Local product -> still single-qubit unitary
        U1 = RZ @ RX @ RY
        U = np.kron(U, U1)
    return U

In [6]:
import numpy as np
import pennylane as qml

n_qubits = 4
np.random.seed(0)

data_encoding_type = 'angle'
data_encoder = angle_encoding_unitary_mixed

if False:
    if data_encoding_type == 'angle':

        @qml.qnode(qml.device('default.qubit', wires=n_qubits))
        def data_encoder(data):
            qml.AngleEmbedding(data, wires=range(n_qubits), rotation='Z')
            return qml.state()
        
    elif data_encoding_type == 'amplitude':

        @qml.qnode(qml.device('default.qubit', wires=n_qubits))
        def data_encoder(data):
            qml.AmplitudeEmbedding(data, wires=range(n_qubits), normalize=True)
            return qml.state()

data = np.random.rand(n_qubits)

Sn = create_symmetric_group(len(data))

# Assume that the data satisfies Klein-4 symmetry
klein4_generators = []
for idx, elem in enumerate(Sn.elements):
    if elem == (0, 1, 2, 3):  # identity
        klein4_generators.append(idx)
    if elem == (1, 0, 3, 2):  # (1,2)(3,4)
        klein4_generators.append(idx)
    if elem == (2, 3, 0, 1):  # (1,3)(2,4)
        klein4_generators.append(idx)
    if elem == (3, 2, 1, 0):  # (1,4)(2,3)
        klein4_generators.append(idx)

K4 = create_induced_subgroup(Sn, klein4_generators)


Induced subgroup has 4 elements (generated from 4 generators in 1 iterations)


In [7]:
def build_qubit_permutation_unitary(perm: tuple[int, ...]) -> np.ndarray:
    """
    Return the 2^n x 2^n unitary implementing the qubit permutation 'perm',
    where perm[i] = target position of qubit i.
    Maps |b_0 b_1 ... b_{n-1}> -> |b_{p^{-1}(0)} b_{p^{-1}(1)} ...>.
    We define action so that bit originally at position i moves to position perm[i].
    """
    n = len(perm)
    dim = 2 ** n
    U = np.zeros((dim, dim), dtype=complex)
    # Precompute inverse permutation to map basis indices cleanly
    inv = [0]*n
    for i, p in enumerate(perm):
        inv[p] = i
    for basis_in in range(dim):
        bits = [(basis_in >> k) & 1 for k in range(n)]
        # Reorder bits according to inverse so that new position j gets old bit from inv[j]
        permuted_bits = [bits[inv[j]] for j in range(n)]
        basis_out = sum(permuted_bits[k] << k for k in range(n))
        U[basis_out, basis_in] = 1.0
    return U

def derive_unitaries_angle_embedding_analytic(group) -> dict[int, np.ndarray]:
    """
    Analytic induced representation for permutation group acting on indices of AngleEmbedding.
    Assumes group.elements is an iterable of permutations as tuples (p0,p1,...,p_{n-1}),
    where element maps index i -> perm[i].
    """
    unitaries = {}
    for idx, perm in enumerate(group.elements):
        U_s = build_qubit_permutation_unitary(perm)
        unitaries[idx] = U_s
    return unitaries

In [8]:
from induced_representation import derive_unitaries_from_equivariance

unitaries = derive_unitaries_angle_embedding_analytic(K4)  # derive_unitaries_from_equivariance(data, K4, data_encoder, n_qubits=n_qubits)

In [9]:
for s, U in unitaries.items():
    print(np.allclose(U.conj().T @ U, np.eye(U.shape[0])))  # Check unitarity

True
True
True
True


In [10]:
for s, U_s in unitaries.items():
    x_transformed = K4.apply(s, data)
    U_Vsx = data_encoder(x_transformed)
    U_x = data_encoder(data)
    
    error = np.linalg.norm(U_Vsx - U_s @ U_x @ U_s.conj().T, 'fro')
    print(f"Operator equivariance error for s={s}: {error:.2e}")

Operator equivariance error for s=0: 0.00e+00
Operator equivariance error for s=1: 4.51e-16
Operator equivariance error for s=2: 5.65e-16
Operator equivariance error for s=3: 4.88e-16


In [11]:
# Verify that U(V_s[x]) = U_s @ U(x) @ U_s^†
if True:   
    for s, U_s in unitaries.items():
        U_V_sx = data_encoder(K4.apply(s, data))
        U_x = data_encoder(data)
        assert np.allclose(U_V_sx, U_s @ U_x @ U_s.conj().T, atol=1e-10), f"Failed for s={s}"

In [12]:
#Synthesize the unitaries to circuits with qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from qiskit import transpile

if False:
    synthesized_circuits = {}
    for elem, U in unitaries.items():
        qc = QuantumCircuit(n_qubits)
        qc.unitary(Operator(U), range(n_qubits))
        transpiled_qc = transpile(qc, basis_gates=['rx', 'ry', 'rz', 'cx', 'cz', 'swap'], optimization_level=3)
        synthesized_circuits[elem] = transpiled_qc
        print(f"Element {elem}:")
        print(transpiled_qc)

In [13]:
from twirling import apply_twirling_to_generators

#unitaries = {elem: U for elem, (U, _) in unitaries.items()}
ansatz_generators = get_ansatz_generators(ansatz14)  # or ansatz14
twirled_generators = apply_twirling_to_generators(unitaries, ansatz_generators, len(K4.elements), n_qubits)

In [14]:
from qiskit.circuit import Parameter
from qiskit.circuit.library import HamiltonianGate

if False:
  synthesized_circuits = {}
  for key in twirled_generators:
      elem = twirled_generators[key]
      qc = QuantumCircuit(n_qubits)
      # Method 1: Direct exponentiation (exact but expensive)
        # Evolution time parameter
      theta = Parameter('θ')
      H = elem['averaged']
      ham_gate = HamiltonianGate(H, theta, label=f'G_{key}')
      qc.append(ham_gate, range(n_qubits))
      qc_bound = qc.assign_parameters({theta: np.pi/2})
      transpiled_qc = transpile(qc_bound, basis_gates=['rx', 'ry', 'rz', 'cx', 'cz', 'swap'], optimization_level=3)
      synthesized_circuits[key] = transpiled_qc
      fig = transpiled_qc.draw('mpl')
      fig.savefig(f"twirled_generator_{key}.png")

In [15]:
from symmetry_analysis import comprehensive_generator_comparison


comparison_results = comprehensive_generator_comparison(
    twirled_generators, unitaries, len(K4), n_qubits
)

COMPREHENSIVE GENERATOR COMPARISON

Generator 0: RY on wires Wires([0])

📏 DISTANCE METRICS:
  Frobenius distance:  1.732051
  Relative distance:   0.866025
  Operator norm dist:  0.750000

🔄 SYMMETRY METRICS:
  Original max violation:  2.83e+00
  Twirled max violation:   0.00e+00
  Symmetry improvement:    2.83e+00
  Twirled is symmetric:    True

📊 SPECTRAL ANALYSIS:
  Spectral distance:       1.414214
  Entropy change:          -0.518731
  Degeneracy (original):   2
  Degeneracy (twirled):    5

🔲 STRUCTURE METRICS:
  Off-diagonal (original): 1.0000
  Off-diagonal (twirled):  1.0000
  Block diag. improvement: 0.2500
  Sparsity improvement:    -0.1875

⚡ COMMUTATOR ANALYSIS:
  Mean commutator (orig):  2.12e+00
  Mean commutator (twirl): 0.00e+00
  Commutator reduction:    2.12e+00

Generator 1: RY on wires Wires([1])

📏 DISTANCE METRICS:
  Frobenius distance:  1.732051
  Relative distance:   0.866025
  Operator norm dist:  0.750000

🔄 SYMMETRY METRICS:
  Original max violation:  2.83